```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [1]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 100

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 1000

#마지막 세대수 
largest_generation = 500

#보존할 상위값
save_value = 5


In [2]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [3]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [13]:
# 바뀌야됨

def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return (777 - (6 * sp)) * 8

In [5]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [6]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += ScoreFunction(speedlist[k][i][j])


    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [14]:
Score_per_all = Make_Score_per_all(speedlist)
Score_per_all

[6802464.0,
 6761904.0,
 6835440.0,
 6888864.0,
 6938064.0,
 6746208.0,
 6909216.0,
 6653376.0,
 6799152.0,
 6816384.0,
 6708096.0,
 6810432.0,
 7040400.0,
 6874272.0,
 6750912.0,
 6740112.0,
 6849120.0,
 6959424.0,
 6742368.0,
 6722208.0,
 6804960.0,
 6852288.0,
 6942816.0,
 6851040.0,
 7021920.0,
 6895584.0,
 6743280.0,
 6506112.0,
 6955056.0,
 6812160.0]

In [16]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)

print("-"*100)

#하루마다의 점수 (이걸로 경쟁)
print(Score_per_all[1])
#열차마다의 점수
#print(Score_per_train[1][2])
#구간마다의 점수
print(ScoreFunction(speedlist[1][2][1]))


30 100 11 8 총개수 :  264000
----------------------------------------------------------------------------------------------------
6761904.0
1992.0


In [23]:
def Score_assessment_group(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation


In [24]:
#평가한 값 
Group_assessment_value = Score_assessment_group(Score_per_all)

#                                         정렬   점수값이 높은 순으로
sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
#상위 save_value 개값        
save_assessmend_value = sortand_Group_assessmend[0:save_value]
# Score_per_all에서 몇번 배열이 상위 5개인지만 
# 리스트 해쉬값만
save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]

print(save_assessmend_num)

print(Score_per_all[save_assessmend_num[1]])

Group_assessment_value

[12, 24, 17, 28, 22]
7021920.0


[[6802464.0, 0.033225923525842596, 0],
 [6761904.0, 0.06625373597631483, 1],
 [6835440.0, 0.09964072732319817, 2],
 [6888864.0, 0.13328866260722616, 3],
 [6938064.0, 0.16717691014244304, 4],
 [6746208.0, 0.2001280571235116, 5],
 [6909216.0, 0.2338753996216899, 6],
 [6653376.0, 0.26637311841368594, 7],
 [6799152.0, 0.2995828648221314, 8],
 [6816384.0, 0.3328767791312372, 9],
 [6708096.0, 0.36564177203675063, 10],
 [6810432.0, 0.39890661442473696, 11],
 [7040400.0, 0.43329471144242676, 12],
 [6874272.0, 0.46687137362951675, 13],
 [6750912.0, 0.49984549680630874, 14],
 [6740112.0, 0.5327668685133276, 15],
 [6849120.0, 0.5662206783885902, 16],
 [6959424.0, 0.6002132566084696, 17],
 [6742368.0, 0.6331456475113966, 18],
 [6722208.0, 0.6659795690040803, 19],
 [6804960.0, 0.6992176839807149, 20],
 [6852288.0, 0.7326869676204444, 21],
 [6942816.0, 0.7665984258023615, 22],
 [6851040.0, 0.8000616137166949, 23],
 [7021920.0, 0.8343594471083284, 24],
 [6895584.0, 0.8680402055291042, 25],
 [6743280.

In [25]:
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][1]:
                break
        if Group_assessment_value[i-1][1] < roulette < Group_assessment_value[i][1]:
            break
    return i

In [29]:
Selected_list_in_roulette = []

#룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
for i in range(N-5):
    Selected_list_in_roulette.append(roulettes(Group_assessment_value))

Selected_list_in_roulette += save_assessmend_num


Selected_list_in_roulette

[14,
 29,
 20,
 26,
 8,
 10,
 12,
 8,
 16,
 0,
 6,
 4,
 15,
 9,
 25,
 5,
 21,
 20,
 1,
 1,
 9,
 6,
 1,
 8,
 5,
 12,
 24,
 17,
 28,
 22]

In [78]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = [[[0 for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]

    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        for j in range(One_day_train):
            for k in range(Station_section):
                print(1)
                #교차
                selection = random.random()
                ###############################################################
                if selection < Pc:
                    rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
                    # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값

                    W[i*2][j][k]     = speedlist[i*2][j][1][0:rand]         + speedlist[i*2][j][1][rand:]
                    #W[i*2][j][k]     = speedlist[i*2][j][Selected_list_in_roulette[i*2]][0:rand]         + speedlist[i*2][j][Selected_list_in_roulette[i*2+1]][rand:]
                    #W[i*2+1][j][k] = speedlist[i*2+1][j][Selected_list_in_roulette[i*2+1]][0:rand] + speedlist[i*2+1][j][Selected_list_in_roulette[i*2]][rand:]
                else:
                    W[i*2][j][k]     = speedlist[i*2][j][1][0:rand]         + speedlist[i*2][j][1][rand:]
                    #W[i*2][j][k]     = (speedlist[i*2][j][Selected_list_in_roulette[i*2]])
                    #W[i*2+1][j][k] = (speedlist[i*2+1][j][Selected_list_in_roulette[i*2+1]])
                #변의
                for z in range(2):
                    # Pm * 100 분의 1의 확률 
                    if randint(0, 100 * Pm) == 0:
                        #selection 부분을 변의 1 -> 0  0 -> 1
                        W[i*2+z][j][k][selection] = -1 * (W[i*2+z][j][k][selection] -1)
    return W

In [79]:
New_list = Change_parent(speedlist, Selected_list_in_roulette)

1


IndexError: list index out of range

In [77]:
len(New_list[0][0][0])

8

In [52]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        #전체 값으로 해서 
        Score_per_all = Make_Score_per_all(speedlist)

        #평가한 값 
        Group_assessment_value = Score_assessment_group(Score_per_all)

        #                                         정렬   점수값이 높은 순으로
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
        #상위 save_value 개값        
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        # Score_per_all에서 몇번 배열이 상위 5개인지만 
        # 리스트 해쉬값만
        save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]


        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        Selected_list_in_roulette += save_assessmend_num
        
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        
        New_Score_per_all = Make_Score_per_all(New_list)
        
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_Score_per_all):
            print(i, nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

In [60]:
test_list = []

test_list = Evolution(speedlist, 0)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0,

IndexError: list index out of range

1


IndexError: list index out of range

In [37]:
Www = [[[0 for i in range(2)]for j in range(2)]for k in range(2)]
    # 교차함수 및 변이함수 (자식만들기)
    
print(Www)
for i in range(2):
    for j in range(2):
        for k in range(2):
            Www[i][j][k] = [43,43,0,2,2]
            
Www

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


[[[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]],
 [[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]]]